# Namespacing with Pinecone

## Setup

In [ ]:
import os
import pandas as pd
from pinecone import Pinecone
from dotenv import load_dotenv
from pinecone import ServerlessSpec

load_dotenv()

## Creating an Index

In [ ]:
# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'

# configure client
pc = Pinecone(api_key=api_key)

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

index_name = "pinecone-namespacing"

# Delete index if exists
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# Create an index
pc.create_index(
    name=index_name, 
    dimension=2, 
    metric="euclidean",
    spec=spec
)

# wait for index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)
# Connect to the index
index = pc.Index(index_name)

In [ ]:
# Delete index if exists
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# Create an index
pc.create_index(
    name=index_name, 
    dimension=2, 
    metric="euclidean",
    spec=spec
)

# wait for index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

pc.list_indexes().names()

In [ ]:
# Connect to the index
index = pc.Index(index_name)


## Generate movie data

In [ ]:
df = pd.DataFrame()
df["id"] = ["Wall-E", "Up", "Ratatouille", "Toy Story"]
df["vector"] = [[1., 1.], [2., 2.], [3., 3.], [4., 4.]]
df

## Insert vectors without specifying a namespace

In [ ]:
# Insert vectors without specifying a namespace
index.upsert(vectors=zip(df.id, df.vector))
index.describe_index_stats()

## Insert vectors into a namespace

In [ ]:
romantic_comedies = ["Wall-E", "Ratatouille"]
romcom_df = df[df.id.isin(romantic_comedies)]
romcom_df

In [ ]:
# Insert vectors into a namespace
index.upsert(vectors=zip(romcom_df.id, romcom_df.vector), namespace="romantic-comedy")
index.describe_index_stats()

## Query top-3 results, without a namespace

In [ ]:
query_results = index.query(vector=df[df.id == "Wall-E"].vector[0], top_k=3)
query_results

## Query top-3 results, with a namespace


In [ ]:
query_results = index.query(vector=
    df[df.id == "Wall-E"].vector[0], top_k=3, namespace="romantic-comedy"
)
query_results

In [ ]:
# delete the index
pc.delete_index(index_name)